#Table of Contents
* [Play around with python/teensy serial connection](#Play-around-with-python/teensy-serial-connection)
	* [Open connection and grab some lines from serial port](#Open-connection-and-grab-some-lines-from-serial-port)
	* [Reading values in a loop happens fast??](#Reading-values-in-a-loop-happens-fast??)
	* [Because USB buffer has accumulated lots of unread values](#Because-USB-buffer-has-accumulated-lots-of-unread-values)


In [1]:
%%javascript
IPython.load_extensions('calico-document-tools');

<IPython.core.display.Javascript object>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import division
from __future__ import print_function
import serial
import time

# Play around with python/teensy serial connection

__NOTE__: Teensy is configured to continuously send the sequence 1, 2, 3, ... 98, 99, 100 over and over. Since we use serial.write() on the Teensy, the numbers are sent as strings. Each string is terminated with '/r/n'. <font color=red>__The interval between sending numbers is set to 1 second in the Teensy code__</font>.

## Open connection and grab some lines from serial port

In [2]:
teensy = serial.Serial('/dev/cu.usbmodem213141', 9600)

In [8]:
for i in range(5):
    print(i, teensy.readline())
    #time.sleep(0.01)

0 44

1 45

2 46

3 47

4 48



In [27]:
serial.version

['14', '4', '0']

In [28]:
teensy.baudrate

9600

In [29]:
teensy.getPort()

'/dev/cu.usbmodem213141'

In [30]:
teensy.read()

'4'

In [31]:
teensy.readline()

'\r\n'

In [32]:
teensy.readline()

'5\r\n'

In [33]:
teensy.readline()

'6\r\n'

## Reading values in a loop happens fast??

(i.e., there isn't 1 second between getting numbers). Why is that?

In [34]:
data_temp = []
for i in range(10):
    temp = teensy.readline()
    data_temp.append(int (temp.strip()))
    time.sleep(0.01)
print(data_temp)

[7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


## Because USB buffer has accumulated lots of unread values

As illustrated below, the reason is that the USB buffer is filled with lots of values that have already been sent by the Teensy (1st loop). When we flush the input buffer (2nd loop), the values arrive as expected in 1 second intervals

In [9]:
start = time.time()
end = time.time()
elapsed = end - start
print('   No code: {:.3e} s'.format(elapsed))

start = time.time()
for i in range(5):
    temp = teensy.readline()
    end = time.time()
    elapsed = end - start
    print('readline(): {:.3e} s, {}'.format(elapsed, temp.strip()))
    
print('')
teensy.flushInput()
start = time.time()
for i in range(5):
    temp = teensy.readline()
    end = time.time()
    elapsed = end - start
    print('readline(): {:.3e} s, {}'.format(elapsed, temp.strip()))

   No code: 3.099e-05 s
readline(): 1.180e-04 s
readline(): 2.003e-05 s
readline(): 1.788e-05 s
readline(): 1.693e-05 s
readline(): 1.693e-05 s

readline(): 3.895e-01 s, 74
readline(): 1.389e+00 s, 75
readline(): 2.389e+00 s, 76
readline(): 3.389e+00 s, 77
readline(): 4.389e+00 s, 78


## How to check if serial connection is open

In [10]:
teensy.isOpen()

True

## Close serial connection

In [12]:
teensy.close()
teensy.isOpen()

False